# CT画像の読み込み

各ファイルについて
- candidates.csv  
  CTデータの系列が何であるか、結節候補の座標情報、候補が実際に結束であるか否かのフラグを示す。

- annotations.csv  
  結節フラグが建てられた候補に対して、追加の情報を記載しているデータ。追加情報は結節の大きさ  
  このデータを利用することで、訓練データと検証データの結節の大きさの分散を似たようなものにできる＝確率分布が似る。  
  ⇨大小様々な結束を判別する必要があるため各データで分散は一致していた方が望ましい。  

In [5]:
# 頭に！をつけることでlinuxコマンドを実行できる、行数を数える
!wc -l ./data/candidates.csv

  551066 ./data/candidates.csv


In [58]:
! head -n 2 data/candidates.csv

1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,-56.08,-67.85,-311.92,0


In [34]:
! grep ',1$' ./data/candidates.csv | wc -l

       0


In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('data/candidates.csv')
data.head()

,seriesuid,coordX,coordY,coordZ,class
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-56.08,-67.85,-311.92,0
1,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,53.21,-244.41,-245.17,0
2,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,103.66,-121.80,-286.62,0
3,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-33.66,-72.75,-308.41,0
4,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-32.25,-85.36,-362.51,0


In [6]:
data.shape

(551065, 5)

In [8]:
data.columns

Index(['seriesuid', 'coordX', 'coordY', 'coordZ', 'class'], dtype='object')

In [17]:
# ラベルが１のデータの数と比率を確認
(data['class'] == 1).sum(), (data['class'] == 1).sum()/data['class'].count() 

(1351, 0.0024516164154863764)

In [20]:
! wc -l ./data/annotations.csv

    1187 ./data/annotations.csv


In [22]:
!head data/annotations.csv

seriesuid,coordX,coordY,coordZ,diameter_mm
1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,-128.6994211,-175.3192718,-298.3875064,5.651470635
1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,103.7836509,-211.9251487,-227.12125,4.224708481
1.3.6.1.4.1.14519.5.2.1.6279.6001.100398138793540579077826395208,69.63901724,-140.9445859,876.3744957,5.786347814
1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016233746780170740405,-24.0138242,192.1024053,-391.0812764,8.143261683
1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016233746780170740405,2.441546798,172.4648812,-405.4937318,18.54514997
1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016233746780170740405,90.93171321,149.0272657,-426.5447146,18.20857028
1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016233746780170740405,89.54076865,196.4051593,-515.0733216,16.38127631
1.3.6.1.4.1.14519.5.2.1.6279.6001.100953483028192176989979435275,81.50964574,54.9572186,-150.3464233,10.36232088
1.3.6.1.4.1.14519.5.2.1.6279.6001.10268196240

In [44]:
# 対応しているもの同士を確認すると、座標に若干のずれがある。原因は結束の大きさが存在するため
# 修正する労力と、それに対する効果が見込めないため、ずれがあるまま分析する
! grep 1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860 ./data/annotations.csv

# ! grep '1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860.*' ./data/candidates.csv
data.loc[(data['seriesuid'] == '1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860') & (data['class'] == 1), :]

1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,-128.6994211,-175.3192718,-298.3875064,5.651470635
1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,103.7836509,-211.9251487,-227.12125,4.224708481


,seriesuid,coordX,coordY,coordZ,class
13,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,104.164804,-211.685591,-227.011364,1
78,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-128.940000,-175.040000,-297.870000,1


### candidates.csvとannotations.csvの２つを繋ぎ合わせる関数を作成する

In [46]:
from collections import namedtuple

CandidateInfoTuple = namedtuple('CandidateInfoTuple',
                               'isNodule_bool, diameter_mm, series_uid, center_xyz',
                               )

In [47]:
CandidateInfoTuple

__main__.CandidateInfoTuple

サニタイズ（不要部分の除去）、クレンジング（整形）

In [66]:
from dsets import getCandidateInfoList, getCt, LunaDataset
# import vis

ModuleNotFoundError: No module named 'util'